In [2]:
install.packages("tidyquant")
install.packages("PerformanceAnalytics")


下载的二进制程序包在
	/var/folders/23/2pq3q16d1nn_mvwz51ct825w0000gq/T//Rtmpb228yQ/downloaded_packages里


Warning message in download.file(url, destfile, method, mode = "wb", ...):
“downloaded length 2145125 != reported length 2293093”
Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/bin/macosx/big-sur-arm64/contrib/4.4/PerformanceAnalytics_2.0.8.tgz': Timeout of 60 seconds was reached”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  download from 'https://cran.r-project.org/bin/macosx/big-sur-arm64/contrib/4.4/PerformanceAnalytics_2.0.8.tgz' failed


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“下载程序包‘PerformanceAnalytics’时出了问题”


In [2]:
# --- 步骤 1: 加载必要的 R 包 ---
library(tidyquant)
library(PerformanceAnalytics)
library(dplyr)

# --- 步骤 2: 定义分析参数 ---
ticker_symbol     <- "QQQ"
start_date        <- "2023-01-01"
end_date          <- Sys.Date()
annual_risk_free_rate <- 0.02  # 年化无风险利率 (2%)
trading_days_per_year <- 252   # 年化因子


# 将年化无风险利率转换为日度无风险利率，以匹配我们的日度回报率数据
daily_risk_free_rate <- annual_risk_free_rate / trading_days_per_year


# --- 步骤 3: 从雅虎财经获取历史价格数据 ---
cat("--- 正在下载 ", ticker_symbol, " 从 ", start_date, " 到 ", format(end_date, "%Y-%m-%d"), " 的数据... ---\n")
qqq_prices <- tq_get(ticker_symbol, 
                     from = start_date, 
                     to = end_date)

print(head(qqq_prices))
print(tail(qqq_prices))

# --- 步骤 4: 计算每日回报率 ---
cat("\n--- 正在计算每日对数回报率... ---\n")
qqq_returns <- qqq_prices %>%
  tq_transmute(select     = adjusted, 
               mutate_fun = periodReturn, 
               period     = "daily", 
               type       = "log",
               col_rename = "log.returns")

# --- 步骤 5: 将数据转换为 XTS 格式 ---
cat("\n--- 正在将数据框转换为 XTS 时间序列对象... ---\n")
qqq_returns_xts <- tk_xts(qqq_returns, 
                          select   = log.returns, 
                          date_var = date)
print(qqq_returns_xts)
# --- 步骤 6: 计算年化夏普比率 ---
cat("\n--- 正在计算年化夏普比率 (年化无风险利率 = ", annual_risk_free_rate * 100, "%)... ---\n")
annualized_sharpe_ratio <- SharpeRatio.annualized(qqq_returns_xts, 
                                                  Rf = daily_risk_free_rate, # 使用正确的日度利率
                                                  scale = trading_days_per_year)

# --- 步骤 7: 打印最终结果 ---
cat("\n======================================================\n")
cat("                最终分析结果\n")
cat("======================================================\n")
print(paste("分析标的:", ticker_symbol))
print(paste("时间范围:", start_date, "至", format(end_date, "%Y-%m-%d")))
print(paste("年化无风险利率:", annual_risk_free_rate * 100, "%"))
cat("------------------------------------------------------\n")
print(annualized_sharpe_ratio)
cat("======================================================\n")

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

── Attaching core tidyquant packages ─────────────────────────────────────────────────── tidyquant 1.0.11 ──
✔ PerformanceAnalytics 2.0.8      ✔ TTR                  0.24.4
✔ quantmod             0.4.26     ✔ xts                  0.14.1
── Conflicts ────────────────────────────────────────────────────────────────────── tidyquant_conflicts() ──
✖ zoo::as.Date()                 masks base::as.Date()
✖ zoo::as.Date.numeric()         masks base::as.Date.numeric()
✖ PerformanceAnalytics::legend() masks graphics::legend()
✖ quantmod::summary()            masks base::summary()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function

--- 正在下载  QQQ  从  2023-01-01  到  2025-10-25  的数据... ---


Warning message:
“x = 'QQQ', get = 'stock.prices': Error in getSymbols.yahoo(Symbols = "QQQ", env = <environment>, verbose = FALSE, : Unable to import “QQQ”.
无法打开链结
”


[1] NA
[1] NA

--- 正在计算每日对数回报率... ---


ERROR: Error in tq_transmute_.default(data = data, select = lazyeval::expr_text(select), : data must be a tibble or data.frame object


### 接下来做什么？解读 1.44 的意义

现在我们有了这个可靠的数字，我们可以来解读它了！

* 夏普比率是什么？ 它衡量的是“风险调整后回报”。简单来说，它回答了这样一个问题：“我每承担一单位的风险（波动性），能获得多少超越无风险利率的回报？”
* 1.44 是好是坏？
  - 夏普比率 > 1 通常被认为是良好的。
  - 夏普比率 > 2 通常被认为是非常好的。
  - 夏普比率 > 3 通常被认为是卓越的。

所以，在 2023-01-01 至 2025-08-20 这个时间段内，QQQ 表现出了良好的风险调整后回报。

In [ ]:
library(showtext) 
library(sysfonts)

font_add_google("Noto Sans SC", "NotoSansSC") # 从 Google Fonts 下载并添加思源黑体（简体中文）
showtext_auto()
theme_set(theme_minimal(base_family = "NotoSansSC", base_size = 12)) 

### 获取 A 股数据


* 上海证券交易所 (Shanghai Stock Exchange): 在 6 位代码后加上 `.SS`
* 深圳证券交易所 (Shenzhen Stock Exchange): 在 6 位代码后加上 `.SZ`


In [ ]:
# 加载必要的包
library(tidyquant)
library(ggplot2)
library(dplyr)

# 定义我们要获取的 A 股股票代码 (使用雅虎财经格式)
a_share_tickers <- c("600519.SS", "000858.SZ")

# 定义时间范围
start_date <- "2023-01-01"
end_date <- Sys.Date()

# 使用 tq_get() 获取数据
cat("--- 正在从雅虎财经下载 A 股数据... ---\n")
a_share_prices <- tq_get(a_share_tickers,
                         from = start_date,
                         to = end_date)

# 查看一下下载的数据结构
print(head(a_share_prices))
print(tail(a_share_prices))

# 使用 ggplot2 可视化收盘价走势
cat("\n--- 正在生成股价走势图... ---\n")
a_share_prices %>%
  # 我们给股票代码起个更友好的名字
  mutate(company = case_when(
    symbol == "600519.SS" ~ "贵州茅台 (Kweichow Moutai)",
    symbol == "000858.SZ" ~ "五粮液 (Wuliangye)",
    TRUE ~ symbol
  )) %>%
  ggplot(aes(x = date, y = close, color = company)) +
  geom_line(linewidth = 1) +
  labs(
    title = "中国 A 股股价走势",
    subtitle = paste("从", start_date, "到", format(end_date, "%Y-%m-%d")),
    x = "日期 (Date)",
    y = "收盘价 (RMB)",
    color = "公司 (Company)"
  ) +
  theme_tq() +
  scale_color_tq()

## 获取 A 股大盘指数


* 上证综合指数 (SSE Composite Index): `000001.SS`
* 深证成份指数 (SZSE Component Index): `399001.SZ`
* 沪深300指数 (CSI 300 Index): `000300.SS`


In [ ]:
shanghai_composite <- tq_get("000300.SS", from = "2023-01-01")
print(head(shanghai_composite))
print(tail(shanghai_composite))

In [ ]:
library(tidyquant)
library(ggplot2)
library(dplyr)

# 定义我们要获取的 A 股股票代码 (使用雅虎财经格式)
a_share_tickers <- c("000001.SS", "399001.SZ", "000300.SS")

# 定义时间范围
start_date <- "2023-01-01"
end_date <- Sys.Date()

# 使用 tq_get() 获取数据
cat("--- 正在从雅虎财经下载 A 股数据... ---\n")
a_share_prices <- tq_get(a_share_tickers,
                         from = start_date,
                         to = end_date)

# 查看一下下载的数据结构
print(head(a_share_prices))
print(tail(a_share_prices))

# 使用 ggplot2 可视化收盘价走势
cat("\n--- 正在生成股价走势图... ---\n")
a_share_prices %>%
  # 我们给股票代码起个更友好的名字
  mutate(company = case_when(
    symbol == "000001.SS" ~ "上证综指",
    symbol == "399001.SZ" ~ "深证成指",
    symbol == "000300.SS" ~ "沪深300",
    TRUE ~ symbol
  )) %>%
  ggplot(aes(x = date, y = close, color = company)) +
  geom_line(linewidth = 1) +
  labs(
    title = "中国 A 股股价走势",
    subtitle = paste("从", start_date, "到", format(end_date, "%Y-%m-%d")),
    x = "日期 (Date)",
    y = "收盘价 (RMB)",
    color = "公司 (Company)"
  ) +
  theme_tq() +
  scale_color_tq()